# RNN 기반 텍스트 생성 (언어모델)

In [1]:
corpus = ['경마장에 있는 말이 뛰고 있다',
        '그의 말이 법이다',
        '가는 말이 고와야 오는 말이 곱다']

## 1. 데이터 전처리

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer


# tokenizing해서 indexing -> 단어 수 확인
# 데이터를 준비하는 과정
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
vocab_size=len(tokenizer.word_index)
max_feature=vocab_size+1
print(f'단어 집합의 크기:{vocab_size}')
print(f'max_feature:{max_feature}')


단어 집합의 크기:11
max_feature:12


In [21]:
# 언어 모델용 학습 데이터 만들기
#print(tokenizer.word_index)

sequence=list()
for sent in corpus:
    #integer encoding
    indexed_sent = tokenizer.texts_to_sequences([sent])[0]
    print(f'전체문장:{indexed_sent}')

    #두번째 단어까지부터 시작해서 한단어씩 추가해서  학습데이터 생성
    for i in range(1, len(indexed_sent)):
        sequences=indexed_sent[:i+1]
        print(sequence)
        sequence.append(sequence)

print(f'학습 데이터 수:{len(sequences)}')
#앞에 패딩을 넣을 것이다.#패딩을 앞에다가 넣는 이유는?
#short term 
#뒤에 반영하고싶은게 많이 남아있음 
#셀스테이트 
#입력과 정답을 구분 
#1.integer encoding-> 문장길이까지 가져온다. range1부터 시작 
#2.padding 
#3.입력 정답 

전체문장:[2, 3, 1, 4, 5]
[]
[[...]]
[[...], [...]]
[[...], [...], [...]]
전체문장:[6, 1, 7]
[[...], [...], [...], [...]]
[[...], [...], [...], [...], [...]]
전체문장:[8, 1, 9, 10, 1, 11]
[[...], [...], [...], [...], [...], [...]]
[[...], [...], [...], [...], [...], [...], [...]]
[[...], [...], [...], [...], [...], [...], [...], [...]]
[[...], [...], [...], [...], [...], [...], [...], [...], [...]]
[[...], [...], [...], [...], [...], [...], [...], [...], [...], [...]]
학습 데이터 수:6


In [23]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 구하기
maxlen=0
for sequence in sequences:
    if len(sequence)>maxlen:
        maxlen = len(sequence)

#maxlen = max([len(sequence)for sequence in sequences])
print(maxlen)

TypeError: object of type 'int' has no len()

In [ ]:
# 학습 데이터 padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences=pad_sequences(sequences, maxlen=maxlen)
print(padded_sequences)

In [ ]:
# 입력 데이터와 정답 데이터 분리
import numpy as np 
np_sequences=np.array(padded_sequences)
X = np_sequences[:,:-1]
y=np_sequences[:,-1]
print(X)
print(y)

In [ ]:
# 정답 데이터 단어 11개를 카테고리로 one-hot-encoding
from tensorflow.keras.utils import to_categorical
y=to_categorical(y)
print(y)

## 2. 모델 학습

In [ ]:
# 모델 구축(레이어 설계+ activation)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

input_units = max_features
embedding_dim = 10
rnn_units = 32
output_units = max_features

model = Sequential()
model.add(Embedding(input_units, embedding_dim))
#Embedding(input_units, embedding_dim, input_length=max_len)

# 모델을 나중에 사용할 때(배포할 때), 입력 길이를 자동으로 인식하지 못할 수 있어서
# input_length를 미리 정해줘야 함 

model.add(SimpleRNN(rnn_units))
model.add(Dense(output_units, activation='softmax'))
model.summary()


In [ ]:
# 모델 컴파일(학습설계)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 모델 학습
model.fit(X, y, epochs=200, verbose=1)

## 3. 첫 단어 입력 후 문장 생성

In [ ]:
#단어를 입력하고, 반복할 회수를 주면 문장 생성
# 모델, 토크나이저, 현재 단어, 반복할 횟수
def sentence_generation(model, tokenizer, init_word, n): 
    current_word = init_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # Integer Encoding
       sequence= tokenizer.texts_to_sequences([current_word])
        # Padding
       padded_sequence=pad_sequences(sequence, maxlen=maxlen-1)

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
       result = model.predict(padded_sequence, verbose=0)
       print(result)
        #print(result)
       result_index=np.argmax(result)
        print(result_index)
        # 예측한 인덱스의 단어 가져오기
       word=tokenizer.index_word[result_index]
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                    

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
       current_word = current_word+''+word
        # 예측 단어를 문장에 저장
       sentence = sentence+''+word

    return init_word + sentence
sentence_generation(model, tokenizer, '경마장에', 4)
  

In [ ]:
sentence_generation(model, tokenizer, input('시작 단어열 입력:'),int(input('생성단어수:')))

In [ ]:
#단어를 입력하고, 반복할 회수를 주면 문장 생성
# 모델, 토크나이저, 현재 단어, 반복할 횟수
def sentence_generation(model, tokenizer, init_word, n): 
    current_word = init_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # Integer Encoding
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        # Padding
        padded = pad_sequences([encoded], maxlen=max_len-1) #, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(padded, verbose=0)
        #print(result)
        result_index = np.argmax(result, axis=1)

        # 예측한 인덱스의 단어 가져오기
        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result_index:                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

sentence_generation(model, tokenizer, '경마장에', 4)